In [1]:
import sys
sys.path.append("../")
from db.miflora import MIFloraDB
from db.core import SpecimenSearchRecord
import json
import sqlparse
from sqlalchemy.orm.query import Query
from sqlalchemy import create_engine, func
from db.miflora import SpecimenRecords, MIFlora2010, FamilyGenus, MIFLORA_BASE, PageTextSpecies

def print_query(query):
    print(sqlparse.format(str(query), reindent=True, keyword_case='upper'))

In [2]:
credentials_file = "../mif_creds.p"
miflora_db = MIFloraDB(credentials_file=credentials_file)
miflora_db.kickstart()

In [ ]:
def list_locations(plant_id):
    q = miflora_db.session.query(mf.CountySpeciesLookup.place_name, mf.CountySpeciesLookup.place_name_type)
    q = q.join(mf.CoSpecies,
               mf.CountySpeciesLookup.place_name_db == mf.CoSpecies.county)
    q = q.filter(mf.CoSpecies.plant_id == plant_id)
    q = q.distinct().order_by(mf.CountySpeciesLookup.island_county_sort, mf.CountySpeciesLookup.place_name_type)
    
    print_query(q)
    
    return q.all()

In [ ]:
def list_synonyms(plant_id):
    q = miflora_db.session.query(mf.Synonyms.syn_species).filter(mf.Synonyms.plant_id == plant_id)
    q = q.distinct().order_by(mf.Synonyms.plant_id)
    
    print_query(q)
    
    return q.all()

In [ ]:
def get_specimen_record(specimen_id):
    q = miflora_db.session.query(SpecimenRecords, FamilyGenus.family, MIFlora2010.plant_id)
    q = q.outerjoin(MIFlora2010, SpecimenRecords.extra_numerical1 == MIFlora2010.plant_id)
    q = q.outerjoin(FamilyGenus, SpecimenRecords.genus == FamilyGenus.genus)
    q = q.filter(SpecimenRecords.identity == specimen_id)
    
    print_query(q)
    
    return q.all()

In [ ]:
def list_table_columns(table_name):
    columns = list()
    
    for column_name in table_name.__dict__.keys():
        if not column_name.startswith('_'):
            columns.append(column_name)

    return columns

In [ ]:
def get_flora_species_text(species_id):
    search_query = miflora_db.session.query(PageTextSpecies.text)
    db_results = search_query.filter(PageTextSpecies.species_id == species_id).first()
    
    return db_results

In [4]:
miflora_db.get_flora_species_text(267)

'<p>Roadsides, fields, and gravelly banks. The oldest Michigan collection seen is from Eaton Co. in 1924. This Eurasian species is locally introduced in North America.</p>\n'